# Naver movie 개봉 전 review scraping

In [36]:
import re
import math
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

class NaverMovieScraping:
    
    def __init__(self, url):
        self.url = url
        
    def URLcode(self, url):
        self.url = url
        start = re.search('code=[0-9]+',self.url).group()
        code = re.search('[0-9]+',start).group()
        return code 
    
    def ReviewSCORE(self, code):
        
        self.code = code
        url_base = "http://movie.naver.com/movie/bi/mi/point.nhn?code={}".format(str(code))
        html = urlopen(url_base).read().decode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        graph = soup.findAll('span',{'class':'exp_cnt'})
        star = soup.findAll('div', {'class':'sc_area b_star'})
        movie_info = soup.findAll('dl', {'class' : 'info_spec'})
        exp_idx = [elem.text for elem in graph]
        star_idx = [s.text for s in star]
       
        try:
            smile = ''.join(re.findall('[0-9]',exp_idx[0]))
            not_smile= ''.join(re.findall('[0-9]',exp_idx[1])) 
        except IndexError as ex:
            smile = 0
            not_smile = 0
    
        try:
            star_val = star_idx[0][17:21]
        except IndexError as ex:
            star_val = 0
        
        try:
            num = star_idx[0][26:31]
        except IndexError as ex:
            num = 0
            
        try:
            text = [info.get_text().replace('\n', '').replace('\t', '').replace('\r','') for info in movie_info]
                           
        except ValueError as ex:
            text = []
            
        return pd.DataFrame({'보고싶어요' : [smile], 
                         '글쎄요' : [not_smile], 
                         '평점' : [star_val], 
                         '참여인원' : [num],
                         '영화정보' :[text]
                            })
    
    def getActors(self, code):
        
        self.code = code
        url_role = "http://movie.naver.com/movie/bi/mi/detail.nhn?code={}".format(str(code))
        html_role = urlopen(url_role).read().decode('utf-8')
        soup = BeautifulSoup(html_role, 'lxml')
        actors = soup.findAll('a', {'class':'k_name'})
        roles = soup.findAll('p', {'class':'in_prt'})
        
        actor = [act.text for act in actors]
        role = [r.text.replace('\n','') for r in roles]
        
        return pd.DataFrame({ '배우' : [actor[:len(actor)-1]],
                             '역할' : [role],
                             '감독' : [actor[len(actor)-1]]})
    
    def review_page(self, code):
        
        self.code = code
        #url의 경우 따로 개봉전 주소를 넣는다.
        url_base = "http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=before".format(str(code))
        html = urlopen(url_base).read().decode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        
        score_total = soup.findAll('div', {'class' : 'score_total'})
        review_num = [elem.findChildren('em')[1].get_text() for elem in score_total]
    
        # review_num이 1000개를 넘어가면 , 문자를 제거 
        return math.ceil(int(review_num[0].replace(",","")) / 10)
    
    def getReview(self, code, page_num):
    
        page = int(1)
        #count = int(input('page number : '))
        self.page_num = int(page_num) #review의 전체 페이지
    
        reple_list = []
        score_list = []
        good_list = []
        bad_list = []
        self.code = code
        
        while page_num:
        
            #url의 경우 따로 개봉전 주소를 넣는다.
            url_base = "http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={}&type=before&page={}".format(str(code),str(page))
            html = urlopen(url_base).read().decode('utf-8')
            soup = BeautifulSoup(html, 'lxml')
        
            score_result = soup.find('div', {'class' : 'score_result'})
            li_list = score_result.find_all('li')
        
            for li in li_list:
            
                page = int(page)
                reple = li.find('div', {'class' : 'score_reple'}).find('p').get_text()
            
            #별점이 없는 곳은 0점으로 넣는다.
                try:
                    score = li.find('div', {'class' : 'star_score'}).find('em').get_text()
                except Exception as ex:
                    score = 0
                
                good = li.find('div', {'class' : 'btn_area'}).get_text().split('\n')[1]
                bad = li.find('div', {'class' : 'btn_area'}).get_text().split('\n')[2]
            
                reple_list.append(reple)
                score_list.append(score)
                good_list.append(''.join([s for s in str(good) if s.isdigit()]))
                bad_list.append(''.join([s for s in str(bad) if s.isdigit()]))
            
            page_num -= 1
        
            if not page_num:
                break
            
            page += 1
    
        return pd.DataFrame({'score' : score_list, 'reple' : reple_list, 
                             'good' : good_list, 'bad' : bad_list})

### Naver movie review

In [2]:
import os
os.listdir(os.getcwd())
#movie_df = pd.read_csv('영화.csv', engine = 'python')
movie_df = pd.read_csv('Url_6715.csv', encoding="cp949")
len(movie_df)

6715

In [3]:
def movie_score(movie_df):
    
    Review_df= pd.DataFrame()
    
    for i, url in enumerate(movie_df['url']):
        print('url 을 긁었습니다.', 'pos :', i)
        crawler = NaverMovieScraping(url)
        Review_df = pd.concat([Review_df, crawler.ReviewSCORE(crawler.URLcode(url))],ignore_index=True)
        #print([getReview(URLcode(url))[49:50],movie_df['movie'][i]])
    
    return(Review_df)

In [4]:
df1 = movie_score(movie_df[:500])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29
url 을 긁었습니다. pos : 30
url 을 긁었습니다. pos : 31
url 을 긁었습니다. pos : 32
url 을 긁었습니다. pos : 33
url 을 긁었습니다. pos : 34
url 을 긁었습니다. pos : 35
url 을 긁었습니다. pos : 36
url 을 긁었습니다. pos : 37
url 을 긁었습니다. pos : 38
url 을 긁었습니다. pos : 39
url 을 긁었습니다. pos : 40
url 을 긁었습니다. pos : 41
url 을 긁었습니다. pos : 42
url 을 긁었습니다. pos : 43
url 을 긁었습니다. pos : 44
url 을 긁었습니다. pos : 4

In [5]:
df2 = movie_score(movie_df[500:1000]) # 55,59

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29
url 을 긁었습니다. pos : 30
url 을 긁었습니다. pos : 31
url 을 긁었습니다. pos : 32
url 을 긁었습니다. pos : 33
url 을 긁었습니다. pos : 34
url 을 긁었습니다. pos : 35
url 을 긁었습니다. pos : 36
url 을 긁었습니다. pos : 37
url 을 긁었습니다. pos : 38
url 을 긁었습니다. pos : 39
url 을 긁었습니다. pos : 40
url 을 긁었습니다. pos : 41
url 을 긁었습니다. pos : 42
url 을 긁었습니다. pos : 43
url 을 긁었습니다. pos : 44
url 을 긁었습니다. pos : 4

In [6]:
df3 = movie_score(movie_df[1000:1500])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29
url 을 긁었습니다. pos : 30
url 을 긁었습니다. pos : 31
url 을 긁었습니다. pos : 32
url 을 긁었습니다. pos : 33
url 을 긁었습니다. pos : 34
url 을 긁었습니다. pos : 35
url 을 긁었습니다. pos : 36
url 을 긁었습니다. pos : 37
url 을 긁었습니다. pos : 38
url 을 긁었습니다. pos : 39
url 을 긁었습니다. pos : 40
url 을 긁었습니다. pos : 41
url 을 긁었습니다. pos : 42
url 을 긁었습니다. pos : 43
url 을 긁었습니다. pos : 44
url 을 긁었습니다. pos : 4

In [21]:
df4 = movie_score(movie_df[1500:2000])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29
url 을 긁었습니다. pos : 30
url 을 긁었습니다. pos : 31
url 을 긁었습니다. pos : 32
url 을 긁었습니다. pos : 33
url 을 긁었습니다. pos : 34
url 을 긁었습니다. pos : 35
url 을 긁었습니다. pos : 36
url 을 긁었습니다. pos : 37
url 을 긁었습니다. pos : 38
url 을 긁었습니다. pos : 39
url 을 긁었습니다. pos : 40
url 을 긁었습니다. pos : 41
url 을 긁었습니다. pos : 42
url 을 긁었습니다. pos : 43
url 을 긁었습니다. pos : 44
url 을 긁었습니다. pos : 4

In [8]:
df5 = movie_score(movie_df[2000:3000])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29
url 을 긁었습니다. pos : 30
url 을 긁었습니다. pos : 31
url 을 긁었습니다. pos : 32
url 을 긁었습니다. pos : 33
url 을 긁었습니다. pos : 34
url 을 긁었습니다. pos : 35
url 을 긁었습니다. pos : 36
url 을 긁었습니다. pos : 37
url 을 긁었습니다. pos : 38
url 을 긁었습니다. pos : 39
url 을 긁었습니다. pos : 40
url 을 긁었습니다. pos : 41
url 을 긁었습니다. pos : 42
url 을 긁었습니다. pos : 43
url 을 긁었습니다. pos : 44
url 을 긁었습니다. pos : 4

In [20]:
df6 = movie_score(movie_df[3000:3500])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29
url 을 긁었습니다. pos : 30
url 을 긁었습니다. pos : 31
url 을 긁었습니다. pos : 32
url 을 긁었습니다. pos : 33
url 을 긁었습니다. pos : 34
url 을 긁었습니다. pos : 35
url 을 긁었습니다. pos : 36
url 을 긁었습니다. pos : 37
url 을 긁었습니다. pos : 38
url 을 긁었습니다. pos : 39
url 을 긁었습니다. pos : 40
url 을 긁었습니다. pos : 41
url 을 긁었습니다. pos : 42
url 을 긁었습니다. pos : 43
url 을 긁었습니다. pos : 44
url 을 긁었습니다. pos : 4

In [22]:
df6_1 = movie_score(movie_df[3500:4000])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29
url 을 긁었습니다. pos : 30
url 을 긁었습니다. pos : 31
url 을 긁었습니다. pos : 32
url 을 긁었습니다. pos : 33
url 을 긁었습니다. pos : 34
url 을 긁었습니다. pos : 35
url 을 긁었습니다. pos : 36
url 을 긁었습니다. pos : 37
url 을 긁었습니다. pos : 38
url 을 긁었습니다. pos : 39
url 을 긁었습니다. pos : 40
url 을 긁었습니다. pos : 41
url 을 긁었습니다. pos : 42
url 을 긁었습니다. pos : 43
url 을 긁었습니다. pos : 44
url 을 긁었습니다. pos : 4

In [10]:
df7 = movie_score(movie_df[4000:5000])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29
url 을 긁었습니다. pos : 30
url 을 긁었습니다. pos : 31
url 을 긁었습니다. pos : 32
url 을 긁었습니다. pos : 33
url 을 긁었습니다. pos : 34
url 을 긁었습니다. pos : 35
url 을 긁었습니다. pos : 36
url 을 긁었습니다. pos : 37
url 을 긁었습니다. pos : 38
url 을 긁었습니다. pos : 39
url 을 긁었습니다. pos : 40
url 을 긁었습니다. pos : 41
url 을 긁었습니다. pos : 42
url 을 긁었습니다. pos : 43
url 을 긁었습니다. pos : 44
url 을 긁었습니다. pos : 4

In [18]:
df8 = movie_score(movie_df[5000:5500])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29
url 을 긁었습니다. pos : 30
url 을 긁었습니다. pos : 31
url 을 긁었습니다. pos : 32
url 을 긁었습니다. pos : 33
url 을 긁었습니다. pos : 34
url 을 긁었습니다. pos : 35
url 을 긁었습니다. pos : 36
url 을 긁었습니다. pos : 37
url 을 긁었습니다. pos : 38
url 을 긁었습니다. pos : 39
url 을 긁었습니다. pos : 40
url 을 긁었습니다. pos : 41
url 을 긁었습니다. pos : 42
url 을 긁었습니다. pos : 43
url 을 긁었습니다. pos : 44
url 을 긁었습니다. pos : 4

In [23]:
df9 = movie_score(movie_df[5500:])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29
url 을 긁었습니다. pos : 30
url 을 긁었습니다. pos : 31
url 을 긁었습니다. pos : 32
url 을 긁었습니다. pos : 33
url 을 긁었습니다. pos : 34
url 을 긁었습니다. pos : 35
url 을 긁었습니다. pos : 36
url 을 긁었습니다. pos : 37
url 을 긁었습니다. pos : 38
url 을 긁었습니다. pos : 39
url 을 긁었습니다. pos : 40
url 을 긁었습니다. pos : 41
url 을 긁었습니다. pos : 42
url 을 긁었습니다. pos : 43
url 을 긁었습니다. pos : 44
url 을 긁었습니다. pos : 4

In [28]:
title = movie_df["title"]
url = movie_df["url"]

In [29]:
title

0                                 0시의 호텔 (Hotel At 00:00)
1                      101번째 프로포즈 (The 101st Proposition)
2                           10대의 반항 (Revolt Of The Teens)
3                                                 10대의 영광
4                                        10분 (10 Minutes)
5                                         10억 (A Million)
6                                                     11月
7                           125 전승철 (125 JEON Seung-Chul)
8                                                12인의 하숙생
9                                                  13세 소년
10                                                13월의 겨울
11                                                13월의 연정
12                                   15년 3시간 전 (15年 3時間前)
13                                            15소년 우주 표류기
14                                            1724 기방난동사건
15        1789 바스티유의 연인들 (1789 Les Amants De La Bastille)
16                  18 : 우리들의 성장 느와르 (18 - Eighteen Noir)
17            

In [30]:
data = pd.concat([df1,df2,df3,df4,df5,df6,df6_1,df7,df8,df9], axis=0)
#data.index = [i for i in range(len(data))]
#movie_df["영화명"]
data["title"] = title

In [31]:
data["url"] = url

In [33]:
data.index = [i for i in range(len(data))]

In [35]:
data.to_csv("score_6715.csv")

In [71]:
data["id"] = movie_df["영화명"]
data["y"] = movie_df["Y (손익분기점)"]

In [72]:
len(data)

84

In [73]:
def movie_score(movie_df):
    
    Review_df= pd.DataFrame()
    
    for i, url in enumerate(movie_df['URL']):
        print('url 을 긁었습니다.', 'pos :', i)
        crawler = NaverMovieScraping(url)
        Review_df = pd.concat([Review_df, crawler.getActors(crawler.URLcode(url))],ignore_index=True)
        #print([getReview(URLcode(url))[49:50],movie_df['movie'][i]])
    
    return(Review_df)

In [74]:
actor1 = movie_score(movie_df[:30])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29


In [75]:
actor2 = movie_score(movie_df[30:60])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23
url 을 긁었습니다. pos : 24
url 을 긁었습니다. pos : 25
url 을 긁었습니다. pos : 26
url 을 긁었습니다. pos : 27
url 을 긁었습니다. pos : 28
url 을 긁었습니다. pos : 29


In [76]:
actor3 = movie_score(movie_df[60:])

url 을 긁었습니다. pos : 0
url 을 긁었습니다. pos : 1
url 을 긁었습니다. pos : 2
url 을 긁었습니다. pos : 3
url 을 긁었습니다. pos : 4
url 을 긁었습니다. pos : 5
url 을 긁었습니다. pos : 6
url 을 긁었습니다. pos : 7
url 을 긁었습니다. pos : 8
url 을 긁었습니다. pos : 9
url 을 긁었습니다. pos : 10
url 을 긁었습니다. pos : 11
url 을 긁었습니다. pos : 12
url 을 긁었습니다. pos : 13
url 을 긁었습니다. pos : 14
url 을 긁었습니다. pos : 15
url 을 긁었습니다. pos : 16
url 을 긁었습니다. pos : 17
url 을 긁었습니다. pos : 18
url 을 긁었습니다. pos : 19
url 을 긁었습니다. pos : 20
url 을 긁었습니다. pos : 21
url 을 긁었습니다. pos : 22
url 을 긁었습니다. pos : 23


In [97]:
actor = pd.concat([actor1, actor2, actor3])
len(actor)

84

In [109]:
a = pd.concat([data, actor], axis=1)

In [30]:
movie_score_3 = new_movie_df

In [2]:
import pandas as pd
score = pd.read_csv("score_1213.csv", encoding="cp949")
del score["Unnamed: 0"]

In [3]:
import pandas as pd
url = pd.read_csv("url_1213.csv", encoding="cp949")
del url["Unnamed: 0"]

In [8]:
url["y"] = score["y"]

In [14]:
url

,title,url,y
0,5914,/movie/bi/mi/basic.nhn?code=90812,0.0
1,"<강(江),원래> 초이스 :: 강길 (River Road)",/movie/bi/mi/basic.nhn?code=85421,0.0
2,10억 (A Million),/movie/bi/mi/basic.nhn?code=52072,1.0
3,18 : 우리들의 성장 느와르 (18 - Eighteen Noir),/movie/bi/mi/basic.nhn?code=102431,0.0
4,19 (Nineteen),/movie/bi/mi/basic.nhn?code=53723,0.0
5,19 (Nineteen),/movie/bi/mi/basic.nhn?code=53723,0.0
6,1987 (1987:When the Day Comes),/movie/bi/mi/basic.nhn?code=158191,0.0
7,1987 (1987:When the Day Comes),/movie/bi/mi/basic.nhn?code=158191,1.0
8,1990년 자유부인 (1990年 自由夫人),/movie/bi/mi/basic.nhn?code=12654,0.0
9,1번가의 기적 (Miracle On 1st Street),/movie/bi/mi/basic.nhn?code=61103,1.0


## Reivew 크롤링

In [15]:
def movie_review(movie_df):
    
    Review_df= pd.DataFrame()
    id = pd.DataFrame(movie_df["title"])
    for i, url in enumerate(movie_df['url']):
        print('url 을 긁었습니다.', 'pos :', i)
        crawler = NaverMovieScraping(url)
        URL = crawler.URLcode(url)
        page = crawler.review_page(URL)
        print(page)
        review = crawler.getReview(URL,page)
        review["id"] = movie_df["title"][i]
        review['y'] = movie_df["y"][i]
        Review_df = pd.concat([Review_df, review],ignore_index=True)
    return(Review_df)

In [16]:
review1 = movie_review(df[])

url 을 긁었습니다. pos : 0
0
url 을 긁었습니다. pos : 1
1
url 을 긁었습니다. pos : 2
28
url 을 긁었습니다. pos : 3
4
url 을 긁었습니다. pos : 4
40


In [40]:
review = pd.concat([review1,review2,review3,review4,review5,review6,review7,review8,review9,review10,
                   review11,review12,review13,review14,review15,])

In [117]:
review1.to_csv("new_Review.csv")

In [ ]:
score = pd.read_csv()